In [362]:
import pandas as pd
import os
import numpy as np

In [363]:
BASE_DIR = os.path.abspath('')
# INPUT_ADD = "data_01.xlsx"
INPUT_ADD = "data_02.xlsx"
result_file_name = "data_02.json"

In [364]:
# reading sets 

In [365]:
sets_data= pd.read_excel(
     io = os.path.join(BASE_DIR, INPUT_ADD),
     sheet_name = 'Sets',
     engine='openpyxl'
)


In [366]:
p_dataframe = sets_data['p']
p_dataframe = p_dataframe.dropna()
p_dataframe = p_dataframe.astype(int)

c_dataframe = sets_data['c']
c_dataframe = c_dataframe.dropna()
c_dataframe = c_dataframe.astype(int)

j_dataframe = sets_data['j']
j_dataframe = j_dataframe.dropna()
j_dataframe = j_dataframe.astype(int)

d_dataframe = sets_data['d']
d_dataframe = d_dataframe.dropna()
d_dataframe = d_dataframe.astype(int)

h_dataframe = sets_data['h']
h_dataframe = h_dataframe.dropna()
h_dataframe = h_dataframe.astype(int)


In [367]:
import gams
ws = gams.workspace.GamsWorkspace(working_directory = BASE_DIR)
db = ws.add_database()

In [368]:
# adding sets to workspace

In [369]:
j_array = j_dataframe.to_numpy(dtype = str)
j_set = db.add_set('j', 1)
for jp in j_array:
    j_set.add_record(jp)

c_array = c_dataframe.to_numpy(dtype = str)   
c_set = db.add_set('c', 1)
for cp in c_array:
    c_set.add_record(cp)

p_array = p_dataframe.to_numpy(dtype = str)    
p_set = db.add_set('p', 1)
for pp in p_array:
    p_set.add_record(pp)

d_array = d_dataframe.to_numpy(dtype = str)    
d_set = db.add_set('d', 1)
for dp in d_array:
    d_set.add_record(dp)
    

h_array = h_dataframe.to_numpy(dtype = str)
h_set = db.add_set('h', 1)
for hp in h_array:
    h_set.add_record(hp)
    



In [370]:
# read parameters and add to workspace

In [371]:
#reading parameter a

In [372]:
parameter_a = np.zeros(shape = (len(c_array),len(p_array)))
parameter_a_dict = {} 
parameter_a_data = pd.read_excel(
     io = os.path.join(BASE_DIR, INPUT_ADD),
     sheet_name = 'A',
     engine='openpyxl'
)
p_indices = parameter_a_data['p']
p_indices = p_indices.dropna()
p_indices = p_indices.astype(str).to_numpy()

c_indices = parameter_a_data['c']
c_indices = c_indices.dropna()
c_indices = c_indices.astype(str).to_numpy()

a_data = parameter_a_data['a(c,p)']
a_data = a_data.dropna()
a_data = a_data.astype(str).to_numpy()

for c_index, p_index, correlated_data in zip(c_indices, p_indices, a_data):
    parameter_a[int(c_index)-1][int(p_index)-1] = float(correlated_data)         

for i in range(parameter_a.shape[0]):
    for j in range(parameter_a.shape[1]):
        parameter_a_dict.setdefault( (str(i+1), str(j+1) ), parameter_a[i][j])

In [373]:
# adding parameter a to worksapce

In [374]:
a = db.add_parameter_dc('a', [c_set , p_set])
for cp in c_array:
    for pp in p_array :
        a.add_record((cp, pp)).value = parameter_a_dict[(cp,pp)]

In [375]:
# reading parameter t

In [376]:
parameter_t = np.zeros(shape = (len(h_array),len(h_array)))
parameter_t_dict = {} 
parameter_t_data = pd.read_excel(
     io = os.path.join(BASE_DIR, INPUT_ADD),
     sheet_name = 'T',
     engine='openpyxl'
)
h_indices = parameter_t_data['h']
h_indices = h_indices.dropna()
h_indices = h_indices.astype(str).to_numpy()

h_prime_indices = parameter_t_data['h_prime']
h_prime_indices = h_prime_indices.dropna()
h_prime_indices = h_prime_indices.astype(str).to_numpy()


t_data = parameter_t_data['t(h,h_prime)']
t_data = t_data.dropna()
t_data = t_data.astype(str).to_numpy()

for h_index, h_prime_index, correlated_data in zip(h_indices, h_prime_indices, t_data):
    parameter_t[int(h_index)-1][int(h_prime_index)-1] = int(correlated_data)         

for i in range(parameter_t.shape[0]):
    for j in range(parameter_t.shape[1]):
        parameter_t_dict.setdefault( (str(i+1), str(j+1) ), parameter_t[i][j])

In [377]:
#adding parameter t to workspace

In [378]:
t=db.add_parameter_dc('t',[h_set , h_set])
for hp in h_array:
    for hpp in h_array :
        t.add_record((hp, hpp)).value = parameter_t_dict[(hp,hpp)]

In [379]:
# reading parameter n

In [380]:
parameter_n = np.zeros(shape = len(c_array))
parameter_n_dict = {} 
parameter_n_data = pd.read_excel(
     io = os.path.join(BASE_DIR, INPUT_ADD),
     sheet_name = 'N',
     engine='openpyxl'
)
c_indices = parameter_n_data['c']
c_indices = c_indices.dropna()
c_indices = c_indices.astype(str).to_numpy()


n_data = parameter_n_data['n(c)']
n_data = n_data.dropna()
n_data = n_data.astype(str).to_numpy()

for c_index, correlated_data in zip(c_indices, n_data):
    parameter_n[int(c_index)-1] = int(correlated_data)         

for i in range(parameter_n.shape[0]):
    parameter_n_dict.setdefault(str(i+1), parameter_n[i])

In [381]:
#adding parameter n to database

In [382]:
n = db.add_parameter_dc('n', [c_set])
for cp in c_array:
    n.add_record(cp).value = parameter_n_dict[cp]

In [383]:
# reading parameter s

In [384]:
parameter_s = np.zeros(shape = (len(j_array),len(c_array)))
parameter_s_dict = {} 
parameter_s_data = pd.read_excel(
     io = os.path.join(BASE_DIR, INPUT_ADD),
     sheet_name = 'S',
     engine='openpyxl'
)
j_indices = parameter_s_data['j']
j_indices = j_indices.dropna()
j_indices = j_indices.astype(str).to_numpy()

c_indices = parameter_s_data['c']
c_indices = c_indices.dropna()
c_indices = c_indices.astype(str).to_numpy()


s_data = parameter_s_data['s(j,c)']
s_data = s_data.dropna()
s_data = s_data.astype(str).to_numpy()

for j_index, c_index, correlated_data in zip(j_indices, c_indices, s_data):
    parameter_s[int(j_index)-1][int(c_index)-1] = int(correlated_data)         

for i in range(parameter_s.shape[0]):
    for j in range(parameter_s.shape[1]):
        parameter_s_dict.setdefault( (str(i+1), str(j+1) ), parameter_s[i][j])

In [385]:
#adding parameter s to workspace

In [386]:
s = db.add_parameter_dc('s', [j_set , c_set])
for jp in j_array:
    for cp in c_array :
        s.add_record((jp, cp)).value = parameter_s_dict[(jp,cp)]

In [387]:
# reading parameter b

In [388]:
parameter_b = np.zeros(shape = (len(p_array),len(d_array),len(h_array)))
parameter_b_dict = {} 
parameter_b_data = pd.read_excel(
     io = os.path.join(BASE_DIR, INPUT_ADD),
     sheet_name = 'B',
     engine='openpyxl'
)
p_indices = parameter_b_data['p']
p_indices = p_indices.dropna()
p_indices = p_indices.astype(str).to_numpy()

d_indices = parameter_b_data['d']
d_indices = d_indices.dropna()
d_indices = d_indices.astype(str).to_numpy()

h_indices = parameter_b_data['h']
h_indices = h_indices.dropna()
h_indices = h_indices.astype(str).to_numpy()


b_data = parameter_b_data['b(p,d,h)']
b_data = b_data.dropna()
b_data = b_data.astype(str).to_numpy()

for p_index, d_index, h_index, correlated_data in zip(p_indices, d_indices, h_indices, b_data):
    parameter_b[int(p_index)-1][int(d_index)-1][int(h_index)-1] = int(correlated_data)         

for i in range(parameter_b.shape[0]):
    for j in range(parameter_b.shape[1]):
        for k in range(parameter_b.shape[2]):
            parameter_b_dict.setdefault( (str(i+1), str(j+1), (str(k+1) )), parameter_b[i][j][k])

In [389]:
#adding parameter b to workspace

In [390]:
b = db.add_parameter_dc('b', [p_set,d_set,h_set])
for pp in p_array:
    for dp in d_array:
        for hp in h_array :
            b.add_record((pp,dp,hp)).value = parameter_b_dict[(pp,dp,hp)]
    


In [391]:
#define scalars and add to workspace

In [392]:
scalar = db.add_set('sca', 1)
scalar.add_record(str(1))

In [393]:
k = db.add_parameter_dc('k', [scalar])
k.add_record(str(1)).value = 2

dlast = db.add_parameter_dc('dlast', [scalar])
dlast.add_record(str(1)).value = 5
             
hbar = db.add_parameter_dc('hbar', [scalar])
hbar.add_record(str(1)).value = 3

In [394]:
#opt = ws.add_options()
#opt.defines['gdxincname'] = db.name

In [395]:
opt = ws.add_options()
opt.defines["gdxincname"] = db.name

In [396]:
model = ws.add_job_from_file("course_scheduling_final.gms")
model.run(opt, databases = db)

In [397]:
objective_function = {}
for rec in model.out_db['z']:
    objective_function.setdefault('objective',rec.level)
print(objective_function)

{'objective': 1.0}


In [398]:
x = {}
for rec in model.out_db['x']:
    x.setdefault( str( (rec.key(0),rec.key(1),rec.key(2)) ),rec.level)
print(x)

{"('1', '1', '1')": 0.0, "('1', '1', '2')": 0.0, "('1', '1', '3')": 0.0, "('1', '1', '4')": 0.0, "('1', '1', '5')": 0.0, "('1', '1', '6')": 0.0, "('1', '1', '7')": 0.0, "('1', '1', '8')": 0.0, "('1', '1', '9')": 0.0, "('1', '2', '1')": 0.0, "('1', '2', '2')": 0.0, "('1', '2', '3')": 0.0, "('1', '2', '4')": 0.0, "('1', '2', '5')": 0.0, "('1', '2', '6')": 0.0, "('1', '2', '7')": 0.0, "('1', '2', '8')": 0.0, "('1', '2', '9')": 0.0, "('1', '3', '1')": 0.0, "('1', '3', '2')": 0.0, "('1', '3', '3')": 0.0, "('1', '3', '4')": 0.0, "('1', '3', '5')": 0.0, "('1', '3', '6')": 0.0, "('1', '3', '7')": 0.0, "('1', '3', '8')": 0.0, "('1', '3', '9')": 0.0, "('1', '4', '1')": 0.0, "('1', '4', '2')": 0.0, "('1', '4', '3')": 0.0, "('1', '4', '4')": 1.0, "('1', '4', '5')": 0.0, "('1', '4', '6')": 0.0, "('1', '4', '7')": 0.0, "('1', '4', '8')": 0.0, "('1', '4', '9')": 0.0, "('1', '5', '1')": 0.0, "('1', '5', '2')": 0.0, "('1', '5', '3')": 0.0, "('1', '5', '4')": 0.0, "('1', '5', '5')": 0.0, "('1', '5', '6'

In [399]:
w = {}
for rec in model.out_db['w']:
    w.setdefault('w',rec.level)
print(w)

{'w': 7.0}


In [400]:
v= {}
for rec in model.out_db['v']:
    v.setdefault( str( (rec.key(0),rec.key(1))),rec.level)
print(v)

{"('8', '1')": 0.0, "('8', '2')": 0.0, "('8', '3')": 0.0, "('8', '4')": 0.0, "('9', '1')": 0.0, "('9', '2')": 0.0, "('9', '3')": 0.0, "('9', '4')": 0.0, "('10', '1')": 0.0, "('10', '2')": 0.0, "('10', '3')": 0.0, "('10', '4')": 0.0, "('11', '1')": 0.0, "('11', '2')": 0.0, "('11', '3')": 0.0, "('11', '4')": 0.0, "('12', '1')": 0.0, "('12', '2')": 0.0, "('12', '3')": 0.0, "('12', '4')": 0.0, "('13', '1')": 0.0, "('13', '2')": 0.0, "('13', '3')": 0.0, "('13', '4')": 0.0, "('14', '1')": 0.0, "('14', '2')": 0.0, "('14', '3')": 0.0, "('14', '4')": 0.0, "('15', '1')": 0.0, "('15', '2')": 0.0, "('15', '3')": 0.0, "('15', '4')": 0.0, "('16', '1')": 0.0, "('16', '2')": 0.0, "('16', '3')": 0.0, "('16', '4')": 0.0, "('17', '1')": 0.0, "('17', '2')": 0.0, "('17', '3')": 0.0, "('17', '4')": 0.0, "('18', '1')": 0.0, "('18', '2')": 0.0, "('18', '3')": 0.0, "('18', '4')": 0.0, "('19', '1')": 0.0, "('19', '2')": 0.0, "('19', '3')": 0.0, "('19', '4')": 0.0, "('20', '1')": 0.0, "('20', '2')": 0.0, "('20',

In [401]:
y = {}
for rec in model.out_db['y']:
    y.setdefault(str(rec.key(0)),rec.level)
print(y)

{'8': 0.0, '9': 0.0, '10': 0.0, '11': 0.0, '12': 0.0, '13': 0.0, '14': 1.0, '15': 0.0, '16': 0.0, '17': 0.0, '18': 0.0, '19': 0.0, '20': 0.0, '21': 0.0}


In [402]:
data = {}
data.setdefault('objective_function',objective_function)
data.setdefault('x',x)
data.setdefault('w',w)
data.setdefault('v',v)
data.setdefault('y',y)

{'8': 0.0,
 '9': 0.0,
 '10': 0.0,
 '11': 0.0,
 '12': 0.0,
 '13': 0.0,
 '14': 1.0,
 '15': 0.0,
 '16': 0.0,
 '17': 0.0,
 '18': 0.0,
 '19': 0.0,
 '20': 0.0,
 '21': 0.0}

In [403]:
import json
with open(os.path.join(BASE_DIR, result_file_name), "w") as write_file:
    json.dump(data, write_file)